In [2]:
import pandas as pd

# 1. CHARGEMENT DE LA TABLE NLP (Celle-là, c'est bon, je reprends ton chemin)
path_nlp = '/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/processed/political_sentiment_DETAILED.csv'
df_nlp = pd.read_csv(path_nlp)
print("✅ Table NLP chargée.")

# 2. CHARGEMENT DE TA MASTER TABLE SOCIO-ÉCO (C'est ICI qu'il faut modifier !)
# Remplace le texte ci-dessous par le chemin de ton fichier CSV qui contient les comtés
# Astuce : Fais un clic droit sur ton fichier dans VS Code -> "Copy Path"

path_master = '/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/processed/master_table_elections.csv'  # <--- MODIFIE CETTE LIGNE

# Sécurité pour vérifier le chargement
try:
    df_master = pd.read_csv(path_master)
    print(f"✅ Master Table chargée. Taille : {df_master.shape}")
except FileNotFoundError:
    print("❌ ERREUR : Python ne trouve pas le fichier master.")
    print("Vérifie que tu as bien mis le bon nom de fichier (ex: 'data/master_table.csv')")
    # On arrête tout si le fichier n'est pas là
    raise 

# --- 3. PRÉPARATION DE LA TABLE NLP (PIVOT) ---

# A. On isole les Démocrates
df_dem = df_nlp[df_nlp['party'] == 'Democrat'].copy()
cols_to_rename = {col: f"DEM_{col}" for col in df_dem.columns if col not in ['year']}
df_dem = df_dem.rename(columns=cols_to_rename).drop(columns=['DEM_party'])

# B. On isole les Républicains
df_rep = df_nlp[df_nlp['party'] == 'Republican'].copy()
cols_to_rename = {col: f"REP_{col}" for col in df_rep.columns if col not in ['year']}
df_rep = df_rep.rename(columns=cols_to_rename).drop(columns=['REP_party'])

# --- 4. LA FUSION (MERGE) ---

print("Fusion en cours...")

# Petite sécurité : Harmonisation des noms de colonnes (Year -> year)
if 'Year' in df_master.columns:
    df_master = df_master.rename(columns={'Year': 'year'})

# On fusionne sur l'année ('year')
# Note : Si ta master table n'a pas de colonne 'year', dis-le moi !
df_final = pd.merge(df_master, df_dem, on='year', how='left')
df_final = pd.merge(df_final, df_rep, on='year', how='left')

# --- 5. CALCUL D'INTERACTIONS ---
# Différentiel : Si > 0, le Démocrate est plus positif que le Républicain
df_final['SENTIMENT_DIFF'] = df_final['DEM_sentiment_mean'] - df_final['REP_sentiment_mean']

# --- 6. SAUVEGARDE FINALE ---
output_path = '/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/processed/PROJET_USA_FINAL_DATASET.csv'
df_final.to_csv(output_path, index=False)

print(f"\n🚀 C'EST FINI ! Ton dataset complet est ici :")
print(output_path)
print("Aperçu des colonnes ajoutées :")
print(df_final[['year', 'DEM_sentiment_mean', 'REP_sentiment_mean']].head())

✅ Table NLP chargée.
✅ Master Table chargée. Taille : (21742, 15)
Fusion en cours...

🚀 C'EST FINI ! Ton dataset complet est ici :
/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/processed/PROJET_USA_FINAL_DATASET.csv
Aperçu des colonnes ajoutées :
   year  DEM_sentiment_mean  REP_sentiment_mean
0  2000            0.146626            0.131232
1  2000            0.146626            0.131232
2  2000            0.146626            0.131232
3  2000            0.146626            0.131232
4  2000            0.146626            0.131232


In [3]:
# Affiche la liste complète des colonnes
print("--- LISTE DES VARIABLES ---")
print(df_final.columns.tolist())

# Affiche un exemple d'une ligne pour comprendre à quoi ça ressemble
print("\n--- EXEMPLE D'UNE LIGNE (2016) ---")
# On regarde juste les colonnes de sentiment pour un comté au hasard
cols_to_check = ['year', 'party', 'DEM_sentiment_mean', 'REP_sentiment_mean', 'SENTIMENT_DIFF']
# Note : 'party' ici dépend de ta master table d'origine, adapte si besoin
print(df_final.head(1))

--- LISTE DES VARIABLES ---
['year', 'state_po', 'county_name', 'fips', 'DEMOCRAT', 'REPUBLICAN', 'unemployment_rate', 'median_income', 'poverty_rate', 'public_workers_pct', 'education_bachelors_pct', 'median_age', 'white_pct', 'black_pct', 'hispanic_pct', 'DEM_sentiment_platform', 'DEM_sentiment_speech', 'DEM_subjectivity_platform', 'DEM_subjectivity_speech', 'DEM_sentiment_mean', 'DEM_subjectivity_mean', 'REP_sentiment_platform', 'REP_sentiment_speech', 'REP_subjectivity_platform', 'REP_subjectivity_speech', 'REP_sentiment_mean', 'REP_subjectivity_mean', 'SENTIMENT_DIFF']

--- EXEMPLE D'UNE LIGNE (2016) ---
   year state_po  county_name  fips  DEMOCRAT  REPUBLICAN  unemployment_rate  \
0  2000       AK  DISTRICT 13  2013  0.335012    0.485081                3.8   

   median_income  poverty_rate  public_workers_pct  ...  \
0        72692.0          12.4                 1.9  ...   

   DEM_subjectivity_speech  DEM_sentiment_mean  DEM_subjectivity_mean  \
0                 0.445372    

In [4]:
print(df_final[df_final['year'] == 2020][['year', 'DEM_sentiment_mean', 'REP_sentiment_mean', 'SENTIMENT_DIFF']])

       year  DEM_sentiment_mean  REP_sentiment_mean  SENTIMENT_DIFF
15530  2020            0.129088            0.157135       -0.028047
15531  2020            0.129088            0.157135       -0.028047
15532  2020            0.129088            0.157135       -0.028047
15533  2020            0.129088            0.157135       -0.028047
15534  2020            0.129088            0.157135       -0.028047
...     ...                 ...                 ...             ...
18632  2020            0.129088            0.157135       -0.028047
18633  2020            0.129088            0.157135       -0.028047
18634  2020            0.129088            0.157135       -0.028047
18635  2020            0.129088            0.157135       -0.028047
18636  2020            0.129088            0.157135       -0.028047

[3107 rows x 4 columns]


Normal même partout ? Oui, c'est 100% normal et c'est exactement ce qu'on attendait !

C'est une étape conceptuelle importante à comprendre pour ton modèle économétrique.

1. Pourquoi c'est pareil partout ?

C'est parce que tes variables de sentiment sont des variables Nationales (Temporelles), alors que tes comtés sont des variables Locales (Spatiales).

Logique : Quand Joe Biden fait un discours ou publie son programme en 2020, ce discours est le même pour un habitant du Texas, de New York ou de la Californie. Il n'y a pas un "Programme Démocrate spécifique pour le comté de Miami-Dade".

Technique : Lors du merge sur la colonne year, Excel/Python a pris la valeur unique de 2020 (ex: -0.02) et l'a copiée-collée sur les 3000 lignes de comtés de l'année 2020.

2. Comment ton modèle va-t-il travailler alors ?

C'est là toute la subtilité. Ton modèle ne va pas comparer le Comté A vs Comté B sur le sentiment (puisque c'est le même).

Il va utiliser la variation temporelle (Time-Series variation) :

"Tiens, en 2016 le sentiment était positif (+0.05) et les Démocrates ont gagné X%."

"En 2020, le sentiment est devenu négatif (-0.02), est-ce que le vote a changé en conséquence ?"

C'est ce qu'on appelle mesurer l'impact d'un choc national sur des résultats locaux.

3. L'Astuce de "Pro" pour créer de la variation locale (Le Bonus)

Si tu veux que ta variable sentiment ait un impact différent selon les comtés (ce qui est très fort économétriquement), tu dois créer un Terme d'Interaction dans ta régression.

L'idée est la suivante : Un discours positif a-t-il plus d'impact dans un comté riche que dans un comté pauvre ?

In [7]:
# --- CRÉATION DES TERMES D'INTERACTION ---

# 1. Interaction Chômage (L'hypothèse : Le discours positif marche moins bien si le chômage est haut)
# Remplace 'Unemployment_Rate' par le nom exact de ta colonne dans ta master table !
col_chomage = 'unemployment_rate' 

# On crée la colonne (Sentiment Différentiel * Chômage Local)
df_final['INTERACT_Sentiment_Chomage'] = df_final['SENTIMENT_DIFF'] * df_final[col_chomage]

# 2. (Optionnel) Interaction Revenu (L'hypothèse : Le discours positif marche mieux chez les riches ?)
# Remplace 'Median_Income' par le vrai nom
if 'Median_Income' in df_final.columns:
    df_final['INTERACT_Sentiment_Revenu'] = df_final['SENTIMENT_DIFF'] * df_final['median_Income']

# --- VÉRIFICATION ---
print("Nouvelles colonnes d'interaction créées :")
print(df_final[['year', 'SENTIMENT_DIFF', col_chomage, 'INTERACT_Sentiment_Chomage']].head())

# --- SAUVEGARDE FINALE (On écrase l'ancien fichier pour inclure ces nouvelles colonnes) ---
df_final.to_csv(output_path, index=False)
print("✅ Fichier sauvegardé avec les interactions.")

Nouvelles colonnes d'interaction créées :
   year  SENTIMENT_DIFF  unemployment_rate  INTERACT_Sentiment_Chomage
0  2000        0.015394                3.8                    0.058499
1  2000        0.015394                4.1                    0.063117
2  2000        0.015394                4.6                    0.070814
3  2000        0.015394                2.5                    0.038486
4  2000        0.015394                3.2                    0.049262
✅ Fichier sauvegardé avec les interactions.


In [6]:
print(df_final.columns.tolist())

['year', 'state_po', 'county_name', 'fips', 'DEMOCRAT', 'REPUBLICAN', 'unemployment_rate', 'median_income', 'poverty_rate', 'public_workers_pct', 'education_bachelors_pct', 'median_age', 'white_pct', 'black_pct', 'hispanic_pct', 'DEM_sentiment_platform', 'DEM_sentiment_speech', 'DEM_subjectivity_platform', 'DEM_subjectivity_speech', 'DEM_sentiment_mean', 'DEM_subjectivity_mean', 'REP_sentiment_platform', 'REP_sentiment_speech', 'REP_subjectivity_platform', 'REP_subjectivity_speech', 'REP_sentiment_mean', 'REP_subjectivity_mean', 'SENTIMENT_DIFF']


ajout de la variable d'interaction

Comment lire ce chiffre dans ta régression ?

Quand tu analyseras tes résultats (le coefficient β associé à cette variable) :

Si le coefficient est POSITIF : Cela veut dire que l'effet du sentiment politique est amplifié par le chômage. (Ex: Un discours positif marche encore mieux dans les zones en crise).

Si le coefficient est NÉGATIF : Cela veut dire que l'effet est atténué. (Ex: Dans les zones où le chômage est élevé, les gens se fichent du discours positif, la réalité économique prend le dessus).